<img align="left" src = https://noirlab.edu/public/media/archives/logos/svg/logo250.svg width=250 style="background-color:white; padding:10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact author(s): Christina Williams, Douglas Tucker <br>
Last verified to run: 2025-03-26 <br>
LSST Science Pipelines version: Weekly 2024_50 <br>
Container Size: medium <br>
Targeted learning level: beginner <br>

**Description:** Explore the available measurements of galaxy photometry produced by the LSST pipelines and their applications.

**Skills:** Basic understanding of photometric measurement methods typically used to characterize galaxies.

**LSST Data Products:** objectTable

**Packages:** `lsst.afw`, `astropy`, `photutils`, `scipy.stats`

**Credit:**
This notebook benefitted from earlier exploration of simulated data and notebook development by Melissa Graham and Dan Taranu, helpful discussions with Jim Bosch, and ideas from Douglas Tucker. 

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>.
Questions are welcome as new topics in the 
<a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> 
of the Rubin Community Forum. 
Rubin staff will respond to all questions posted there.

# 1. Introduction

This tutorial will overview some important data products produced by the LSST pipeline for galaxy science. Section 2 will overview the data products (deepCoadd images, how to access them, and some simple image display and manipulation) and the catalog of objects that made from detections found in those coadds (the objectTable). Section 3 will then overview some of the photometry and shape measurements that are produced of galaxies in deepCoadds.


## 1.2 Import packages


`numpy` is a fundamental package for scientific computing with arrays in Python
(<a href="https://numpy.org">numpy.org</a>).

`matplotlib` is a comprehensive library for creating static, animated, and
interactive visualizations in Python 
(<a href="https://matplotlib.org/">matplotlib.org</a>; 
<a href="https://matplotlib.org/stable/gallery/index.html">matplotlib gallery</a>).

`scipy.stats` contains statistical packages, this notebook will use binned_statistic to calculate running medians.

From the `lsst` package, modules for accessing the TAP service, and image display functions from `afw` are imported (<a href="https://pipelines.lsst.io/">pipelines.lsst.io</a>). Also import some geometric functions to help plot photometric apertures.

From the `pyvo` package, import some functions that will enable using the image cutout tool.

Finally, from `astropy` and `photutils` import packages to enable plotting images and drawing shapes on images with WCS information.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

import lsst.geom as geom
import lsst.resources
import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.rsp import get_tap_service
from lsst.afw.fits import MemFileManager
import lsst.afw.geom.ellipses as ellipses
from lsst.afw.image import MultibandExposure
from lsst.daf.butler import Butler

from pyvo.dal.adhoc import DatalinkResults, SodaQuery
import uuid
import gc

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from photutils.aperture import SkyCircularAperture, SkyEllipticalAperture


## 1.3 Define functions and parameters


### 1.3.1 Define some plotting defaults

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

plt.style.use('tableau-colorblind10')


### 1.3.2 Initialize the TAP service & Instantiate the Butler

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

service = get_tap_service("tap")

### 1.3.3 Define a shortcut function to generate an image cutout using the cutout tool
Further information about the cutout tool can be found in DP0.2 tutorial notebook 13a that demonstrates the Rubin image cutout service. 

In [ ]:
def make_image_cutout(tap_service, ra, dec, cutout_size=0.01):
    """Wrapper function to generate a cutout using the cutout tool

   Parameters
    ----------
    tap_service : an instance of the TAP service
    ra, dec : 'float'
        the ra and dec of the cutout center
    cutout_size : 'float', optional
        edge length in degrees of the cutout

   Returns
    -------
    mem : 'object'
        An lsst.afw.fits._fits.MemFileManager object containing the
        cutout image bytes returned by the cutout tool
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

    band = 'i'

    query = "SELECT access_format, access_url, dataproduct_subtype, " + \
            "lsst_patch, lsst_tract, lsst_band, s_ra, s_dec " + \
            "FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_band = " + "'" + str(band) + "' " + \
            "AND CONTAINS(POINT('ICRS', " + str(coord.ra.value) + \
            ", " + str(coord.dec.value) + "), " + \
            "s_region) = 1"

    results = tap_service.search(query)

    dataLinkUrl = results[0].getdataurl()
    auth_session = service._session
    dl = DatalinkResults.from_result_url(dataLinkUrl,
                                         session=auth_session)

    sq = SodaQuery.from_resource(dl,
                                 dl.get_adhocservice_by_id("cutout-sync"),
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
                 spherePoint.getDec().asDegrees() * u.deg,
                 cutout_size * u.deg)
    cutout_bytes = sq.execute_stream().read()
    mem = MemFileManager(len(cutout_bytes))
    mem.setData(cutout_bytes, len(cutout_bytes))

    return mem

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

## 2. Data Products relevant for galaxies

### 2.1  `deepCoadd` images

This section will examine coadded images made up of multiple exposures that have been combined. This is relevant to galaxy science.

Let's start by taking a look at what a full 4k x 4k pixel coadd "patch" image looks like. 

We start by grabbing a `deepCoadd` image and displaying it:

In [ ]:
dataId = {'tract': 4431, 'patch': 17, 'band': 'i'}
datasetType = 'deepCoadd'
coadd = butler.get(datasetType, **dataId)

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(coadd.image)
plt.show()
#remove_figure(fig)

> Figure 8: A `deepCoadd` patch image, displayed in grayscale with scale bar at right, showing a rich galaxy cluster in the lower-left corner.

In [ ]:
wcs = coadd.getWcs()
print(wcs)

The cluster seems to be centered at about (X, Y) ~ (12500, 8500).
Use the "pixelToSky" method of the WCS to get the sky coordinates.

In [ ]:
radec = wcs.pixelToSky(12500, 8500)
cluster_ra, cluster_dec = radec.getRa().asDegrees(), radec.getDec().asDegrees()
print(cluster_ra, cluster_dec)

Now that we have the RA, Dec coordinates of the cluster, we would like to grab a small cutout of the coadded image at this location. To do this, we've defined a user function `make_image_cutout` at the beginning of the notebook](#make_image_cutout). This function extracts a cutout from a deep coadd image at a given RA, Dec position and desired image size.

### 2.2 The objectTable: catalog of static objects from deepCoadd images

Now that we've been able to examine the coadd image and make a cutout, we may be interested in the corresponding catalog of astronomical sources that was extracted from this image by the LSST Science Pipelines. 
The TAP service is the recommended way to retrieve catalog data for a notebook, and there are examples on how to do this in previous [tutorials](https://github.com/rubin-dp0/tutorial-notebooks).

This section demonstrates how to use TAP to retrieve the catalog data within a polygon defined by the corners of the coadd image.

The following cell extracts the XY values of the corners of the cutout image and converts them to RA, Dec, which are then used as spatial constraints in the query below.

In [ ]:

mem = make_image_cutout(service, cluster_ra, cluster_dec, cutout_size=0.005)

cutout_image = ExposureF(mem)
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)


In [ ]:
wcs = cutout_image.getWcs()

x0 = float(cutout_image.getX0())
y0 = float(cutout_image.getY0())
width = cutout_image.getWidth()
height = cutout_image.getHeight()

xcorners = [x0, x0+width, x0+width, x0]
ycorners = [y0, y0, y0+width, y0+width]

ra_corners = []
dec_corners = []

for i in range(len(xcorners)):
    radec = wcs.pixelToSky(xcorners[i], ycorners[i])
    ra_corners.append(radec.getRa().asDegrees())
    dec_corners.append(radec.getDec().asDegrees())

Create a query statement for `Objects` within the bounding box corners.

> **Notice:** You may be familiar with the `BETWEEN ... AND ...` format for ADQL spatial queries, and tempted to use it here to select on RA and Dec ranges. However, the Qserv database does not know how to deal with this construction efficiently, so you are _much_ better off constructing a polygon from the corners of the image as below.

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, x, y, tract, patch " + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " +\
        "POLYGON('ICRS', " + str(ra_corners[0]) + ", " + str(dec_corners[0]) + ", " +\
        str(ra_corners[1]) + ", " + str(dec_corners[1]) + ", " +\
        str(ra_corners[2]) + ", " + str(dec_corners[2]) + ", " +\
        str(ra_corners[3]) + ", " + str(dec_corners[3]) + ")) = 1 AND " +\
        "tract = " + str(dataId['tract']) + " AND patch = " + str(dataId['patch'])
print(query)

Run the query as an asynchronous job, then fetch the results.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
print(len(results))

Convert the results to an Astropy table, then take a look at the table.

In [ ]:
tab = results.to_table()

Display the image cutout, and overplot the sources on the image. Use display buffering to avoid re-drawing the image after each source is plotted.

In [ ]:


fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
with display.Buffering():
    for i in range(len(tab)):
        display.dot('+', tab[i]['x'], tab[i]['y'], ctype=afwDisplay.RED)
        display.dot('o', tab[i]['x'], tab[i]['y'], size=20, ctype='orange')
plt.show()
remove_figure(fig)

# 3. Integrated photometry of galaxies using the LSST pipeline

This section will explore integrated photometry measurements, and provide some guidance for which are optimal for certain science applications using galaxies. 


Photometry is the measurement of how much light is apparent from astronomical sources. The amount of light arriving on the telescope from the object is typically referred to as the flux density, or apparent magnitude (depending on units). Flux density is defined as the amount of energy arriving on the telescope per unit area, per unit time, per unit frequency (or wavelength) of the light.

The LSST Science Pipelines makes a variety of photometric measurements for point-like and extended sources. This notebook will teach the user about the automated photometry measurements that are measured on the deepCoadd images and appear in the Object Catalog as part of the LSST pipelines data products. The focus will be on galaxies (and thus is distinct from the demonstrations in DP0.2 Tutorial Notebooks 12a (about PSF data products) and 12b (a PSF science demo).

Please note that these explanations are specific to DP0.2: the `objectTable` contents and definitions are still subject to change for DP1. The photometry in the catalog are flux densities in units of nano-Jansky [nJy]. 1 Jy = 10^{-23} ergs/s/cm^2/Hz.


## 3.1 Types of photometry measurements in the Object Table

Schema for the object catalog for DP0.2 <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object">is available here</a>.

Numerous photometry measurements are produced by the LSST Pipelines. Two types of photometry are there. The first are total fluxes (see Section 1.1.1), which aim to approximate (or model) all of the light coming from objects. Measurements that approximate total flux are "Composite Model" (cModel) fluxes. The second class of fluxes are measured inside an on-sky aperture but not corrected for flux that may fall outside: thus they are apparent fluxes but do not recover the intrisic (total) flux (see Section 1.1.2). The apparent fluxes are optimized for other purposes, such as for measuring accurate light profiles or accurate colors.

### 3.1.1 Total fluxes


#### Composite Model (CModel) fluxes

Similar in nature to those measured for SDSS (information <a href="https://www.sdss3.org/dr8/algorithms/magnitudes.php#cmodel">available here</a>). 


In short, it is the linear combination of the best fit exponential (disk or D) and de Vaucouleurs (bulge or B) profiles. 

```
<f>_cModelFlux    : Flux from the final cmodel fit. Forced on <f>-band.
<f>_cModelFluxErr : Uncertainty of <f>_cModelFlux
<f>_cModel_flag   : Failure flag for <f>_cModelFlux
```

For most cases the "fixed" cModel photometry (as listed above) are preferred to that measured with more degrees of freedom labeled `<f>_free_cModelFlux`. The difference is that the fixed ones above uses a reference band (recorded as `refBand` in the schema) where the galaxy is well detected to determine the other parameters, which are then fixed when fitting for the flux in the other bands. The `_free_cModelFlux` measurement allows all parameters to be free and independent in each filter. The fixed `_cModelFlux` measurements are generally recommended for galaxy science applications where total flux measurements are needed (e.g. for intrinsic luminosity or mass).

### 3.1.2 Apparent fluxes (i.e. not corrected to the total flux)
 
#### Kron fluxes

A decent summary of Kron fluxes <a href="https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2_6.html">in the NED documentation</a>. The aperture used for the fluxes is 2.5 x R1 where R1 is the luminosity weighted radius (also called "first moment"; Kron et al. 1980).

```
<f>_kronFlux      : Flux from Kron Flux algorithm. Measured on <f> g-band.
<f>_kronFluxErr   : Uncertainty of <f>_kronFlux.
<f>_kronFlux_flag : Failure flag for <f>_kronFlux.
```

The Kron radius, `<f>_kronRad`, is also available. In this case of LSST pipeline output, the Kron flux is not corrected for light that is emitted outside of the Kron aperture. While in many cases it will collect the majority of light, it will not be as accurate as the cModel for science cases requiring total flux.


#### Aperture fluxes
This contains the enclosed flux inside a given aperture (they are not corrected to total fluxes using an aperture correction that accounts for the flux falling outside the aperture). Fixed aperture size refers to the aperture radius in pixels.

```
<f>_ap<pix>Flux     : Flux within <pix>-pixel aperture. Forced on <f>-band.
<f>_ap<pix>FluxErr  : Uncertainty of <f>_ap<pix>Flux.
<f>_ap<pix>FluxFlag : Failure flag for <f>_ap<pix>Flux.
```

For DP0.2, the apertures are 3, 6, 9, 12, 17, 25, 35, 50, and 70 pixels. In the column name, apertures are `03`, `06`, `09`, `12`, and so on. While aperture fluxes are not corrected for the loss outside the aperture, if the aperture size is much larger than the galaxy size then it will approximate the total flux of the galaxy. The general application of these measurements are for measuring radial profiles (see Section 4 below).

#### GaaP fluxes

These are the Gaussian-aperture-and-PSF flux that is defined in <a href="https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract">Kuijken et al. 2008</a>. The main goal of this method is to measure accurate colors while accounting for the different spatial resolution between filters. This is sometimes achieved in other datasets by convolving all images to the largest PSF, but this process of PSF-matching is computationally very time consuming for large images, thus motivating GaaP as a faster alternative. It is not a measure of total flux in a filter. Several apertures are available. For photometric redshifts, and other analysis where accurate colors are important, it is the GaaP fluxes with optimal aperture that should be used. 

**Optimal**

```
<f>_gaapOptimalFlux    : GaaP flux with optimal aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapOptimalFluxErr : Uncertainty of <f>_gaapOptimalFlux.
```


## 3.2 Photometry measurements in objectTable

First, see what is available in the object catalog by querying the `tap_schema` columns, and printing all the parameters available related to "Flux" measured in the i-band (as an example). For clarity, the return also omits errors and flags associated with the photometric measurements outlined in section 1.1. 

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp02_dc2_catalogs.Object'"

results = service.search(query).to_table()

In [ ]:
search_string = 'Flux'
band = 'i_'
exclude1 = 'Err'
exclude2 = 'flag'
for cname in results['column_name']:
    if cname.find(search_string) > -1 and cname.find(band) > -1 and \
       cname.find(exclude1) == -1 and cname.find(exclude2) == -1:

        print(cname)

## 3.3 Compare photometric measurements

### 3.3.1 Identify a galaxy sample to investigate photometry

Below, query the DP0.2 `objectTable` for a selection of photometric measurements. This section will focus on 4 measurements: total flux from cModel (sum of bulge and disk sersic compoents fitted to the galaxy); and apparent fluxes measured as Kron (typically includes more than 90% of intrinsic light), GaaP (which is optimized for measuring accurate colors between bands), and aperture photometry (flux measured inside circles of varying size).

First, focus on a known part of the simulation to contain a galaxy cluster (also seen in DP0.2 notebook tutorial 03a). Limit the search to contain galaxies using the `i_extendedness` flag (which will exclude point sources), and to those objects that are unique in the `objectTable` (by setting `detect_isPrimary` = True). Further, identify objects which have been detected at high signal to noise > 20 and whose photometric measurements have not been flagged as having an issue (`i_kronFlux_flag` or `i_cModel_flag` = 0 means the photometry is ok). 

In [ ]:
query = "SELECT obj.objectId, obj.coord_ra, obj.coord_dec, " + \
        "obj.detect_fromBlend, obj.detect_isIsolated, " + \
        "obj.i_blendedness, obj.i_extendedness, " + \
        "obj.i_kronFlux, obj.i_kronFluxErr, obj.i_kronRad, " + \
        "obj.i_cModelFlux, obj.i_cModelFluxErr, obj.i_gaapOptimalFlux, " + \
        "obj.r_gaapOptimalFlux, obj.g_gaapOptimalFlux, " + \
        "obj.u_gaapOptimalFlux, obj.i_kronFlux_flag, obj.i_cModel_flag, " + \
        "obj.shape_xx, obj.shape_xy, obj.shape_yy, " + \
        "obj.i_ap03Flux, obj.i_ap06Flux, obj.i_ap09Flux, obj.i_ap12Flux, " + \
        "obj.i_ap17Flux, obj.i_ap25Flux, obj.i_ap35Flux, obj.i_ap50Flux " + \
        "FROM dp02_dc2_catalogs.Object AS obj " + \
        "WHERE (obj.detect_isPrimary = 1) AND " + \
        "(obj.i_cModelFlux/obj.i_cModelFluxErr > 20) AND " + \
        "(obj.i_extendedness = 1) AND " + \
        "(obj.i_kronFlux_flag = 0) AND (obj.i_cModel_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS',"+str(cluster_ra)+","+str(cluster_dec)+", 0.1)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Print the results of the search query.

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

Below, store the Kron Radius, which is a good proxy for the size of the galaxy light profile. Then convert the fluxes extracted from the `objectTable` into AB magnitudes

> **Warning:** The following cell will produce warnings for invalid value encountered in log10, which happens if the source flux is negative. This occasionally happens from aperture photometry if the included pixels inside the aperture have negative values and can be safely ignored for this example.

In [ ]:
i_kronRad = tab['i_kronRad']

cmodel_mag = -2.50 * np.log10(tab['i_cModelFlux']) + 31.4
kron_mag = -2.50 * np.log10(tab['i_kronFlux']) + 31.4
gaap_mag = -2.50 * np.log10(tab['i_gaapOptimalFlux']) + 31.4

ap06_mag = -2.50 * np.log10(tab['i_ap06Flux']) + 31.4
ap09_mag = -2.50 * np.log10(tab['i_ap09Flux']) + 31.4
ap17_mag = -2.50 * np.log10(tab['i_ap17Flux']) + 31.4
ap12_mag = -2.50 * np.log10(tab['i_ap12Flux']) + 31.4
ap35_mag = -2.50 * np.log10(tab['i_ap35Flux']) + 31.4

### 2.3.2 Compare aperture photometry to cModel total fluxes

This section will make several plots that shows how the different photometric measurements compare. Note that cModel is the total flux integrated from a model fit to the galaxy light profile, while Kron, GaaP, and aperture photometry are all measures of light within a fixed aperture.  


#### 2.3.2.1 Aperture photometry

Generally, magnitudes measured using aperture photometry in the LSST pipeline are larger (i.e. fainter) than those measured from cModel, becuase the fixed circular aperture systematically underestimates the flux in the galaxy wings (and the lost flux increases as the intrinsic size of the galaxy increases, e.g. as traced by the Kron radius).

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1,
                              width_ratios=[0.8, 0.2], figsize=(10, 6))

ylims = [-1.5, 1.5]
ax.plot(i_kronRad, (cmodel_mag-ap06_mag), '^', alpha=.3,
        label='6-pixel aperture', color='red')
ax.plot(i_kronRad, (cmodel_mag-ap09_mag), 's', alpha=.3,
        label='9-pixel aperture', color='orange')
ax.plot(i_kronRad, (cmodel_mag-ap12_mag), 'o', alpha=.3,
        label='12-pixel aperture', color='green')
ax.plot(i_kronRad, (cmodel_mag-ap17_mag), '.', alpha=.3,
        label='17-pixel aperture', color='blue')

ax2.hist((cmodel_mag-ap17_mag), edgecolor='blue', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap12_mag), edgecolor='green', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap09_mag), edgecolor='orange', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap06_mag), edgecolor='red', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)

ax2.set_ylim(ylims)
ax.axhline(0, linestyle='--')
ax.set_xlabel('Kron Radius [i-band; pixels]')
ax.set_ylabel('cModel mag - Aperture mag')
ax.set_ylim(ylims)
ax.set_xlim([0, 15])
ax.legend()


> Figure 1: A comparison of the difference between cModel photometry and aperture photometry measured by the LSST pipelines for four different aperture sizes as a function of galaxy size (as measured using the Kron radius). The left panel shows the scatter plot of difference in photometry vs Kron radius, and the right panel shows a histogram of these values that demonstrate that larger aperture sizes have photometry that is closer to the cModel.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 6))
ylims = [-1.2, 1.2]
bins = np.arange(16, 27, 1)

ax.plot(cmodel_mag, (cmodel_mag-ap06_mag), '^', alpha=.1,
        label='cModel 6-pix aperture', color='red')
x = cmodel_mag
y = (cmodel_mag-ap06_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='red', lw=2, label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap09_mag), 's', alpha=.1,
        label='cModel 9-pix aperture', color='orange')
x = cmodel_mag
y = (cmodel_mag-ap09_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='orange', lw=2,
        label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap12_mag), 'o', alpha=.1,
        label='cModel 12-pix aperture', color='green')
x = cmodel_mag
y = (cmodel_mag-ap12_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='green', lw=2,
        label='bin median', zorder=10)

ax.plot(cmodel_mag, (cmodel_mag-ap17_mag), '.', alpha=.1,
        label='cModel 17-pix aperture', color='blue')
x = cmodel_mag
y = (cmodel_mag-ap17_mag)
bin_mean, bin_edge, binnum = binned_statistic(x, y,
                                              statistic='median', bins=bins)
binctr = bin_edge[:-1] + (bin_edge[1:]-bin_edge[:-1])/2
ax.plot(binctr, bin_mean, color='blue', lw=2,
        label='bin median', zorder=10)

ax.axhline(0, linestyle='--')
ax.set_xlabel('cModel Magnitude')
ax.set_ylabel('cModel mag - Aperture mag')
ax.set_ylim([-1, 1])
ax.legend()


> Figure 2: A similar comparison of the difference between cModel photometry and aperture photometry measured by the LSST pipelines for four different aperture sizes, this time as a function of cModel magnitude. Running median is included.

These two figures show that the aperture photometry typically under-estimates the flux relative to the total flux estimated using cModel. As expected, there is a general trend for larger apertures to get closer to the total flux from cModel for large galaxies (i.e. whose Kron Radius is larger). There is also a general trend for the aperture photometry to be less discrepant at fainter magnitudes, since faint galaxies tend to be small.




## 3.4 Visualize photometric measurements

Here make a cutout of a large galaxy in the cluster and compare the photometric aperture sizes of the different photometric measurements. The cell below also demonstrates how to reconstruct the Kron aperture using the corresponding shape parameters in the `objectTable`. The position angle `theta` is defined in radians counterclockwise from the x-axis. This cell will plot for an elongated edge-on galaxy (`objectId` of 1909948454470183768) to ensure that the visualization emphasizes the difference between the elliptical Kron aperture to the circular aperture. 

Alternatively, it is possible to pick a different bright object that has large size using the command

`wh = np.where((tab['i_kronRad'] > 8) & (cmodel_mag < 20))[0]`

Note that the pixel scale of the LSST data is 0.2 arcseconds per pixel.

In [ ]:
# Hmm. consider that you are showing Kron here. 

wh = np.where((tab['objectId'] == 1909948454470183768))[0]
indx = 0

arcsec_per_pix = 0.2

mem = make_image_cutout(service, tab['coord_ra'][wh][indx],
                        tab['coord_dec'][wh][indx], cutout_size=0.005)

cutout = ExposureF(mem)

axes = ellipses.Axes(ellipses.Quadrupole(tab['shape_xx'][wh][indx],
                                         tab['shape_yy'][wh][indx],
                                         tab['shape_xy'][wh][indx]))
Rmaj = axes.getA()
Rmin = axes.getB()
theta = axes.getTheta()

plt.subplot(projection=WCS(cutout.getWcs().getFitsMetadata()))
extent = (cutout.getBBox().beginX, cutout.getBBox().endX,
          cutout.getBBox().beginY, cutout.getBBox().endY)

plt.imshow(cutout.image.array, vmin=.1, vmax=0.8,
           extent=extent, origin='lower', cmap='gray')

coord = SkyCoord(ra=tab['coord_ra'][wh][indx]*u.degree,
                 dec=tab['coord_dec'][wh][indx]*u.degree, frame='icrs')

aperture = SkyCircularAperture(coord, r=17 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='r', lw=3, label='17 pixel aperture')

aperture = SkyCircularAperture(coord, r=35 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='g', lw=3, label='35 pixel aperture')

aperture = SkyEllipticalAperture(coord,
                                 2.5 * Rmaj * arcsec_per_pix * u.arcsec,
                                 2.5 * Rmin * arcsec_per_pix * u.arcsec,
                                 theta=(np.pi/2 + theta) * u.rad)

pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='b', lw=3, label='Kron')
plt.legend()


> Figure 5: An i-band image cutout of an example galaxy (grayscale). The photometric apertures are overplotted including Kron (blue), the 17 pixel aperture (red) and the 35 pixel aperture (green). None of the apertures enclose all of the flux visible from the galaxy, which is why the photometry measures less flux than the total flux from cModel.

The above figure shows more clearly that there is some fraction of the galaxy's light excluded by each of these photometric apertures. Further, printing the different photometry measurements illustrates how these assumed apertures will change the measured flux (compared to the total flux measured by cModel, which approximates the total magnitude using sersic models for the light profile).  

# 4. Exercise for the Learner: application of GaaP Photometry to measure galaxy colors

This section demonstrates using GaaP photometry to calculate accurate galaxy colors to identify different types of galaxies. First, define magnitudes from u, g, r, and i band photometry. The second cell will then compare the colors of galaxies that overlap the galaxy cluster with that in the field. In clusters, galaxies tend to be old, red elliptical galaxies and thus exhibit a well defined red sequence in color space. 

For comparison, search for and plot galaxies from a random location that will be dominated by bluer star forming galaxies


In [ ]:

random_RA = 56.7506834813934
random_dec = -31.28892993702273

query = "SELECT obj.objectId, obj.coord_ra, obj.coord_dec, " + \
        "obj.detect_fromBlend, obj.detect_isIsolated, " + \
        "obj.i_blendedness, obj.i_extendedness, " + \
        "obj.i_kronFlux, obj.i_kronFluxErr, obj.i_kronRad, " + \
        "obj.i_cModelFlux, obj.i_cModelFluxErr, obj.i_gaapOptimalFlux, " + \
        "obj.r_gaapOptimalFlux, obj.g_gaapOptimalFlux, " + \
        "obj.u_gaapOptimalFlux, obj.i_kronFlux_flag, obj.i_cModel_flag " + \
        "FROM dp02_dc2_catalogs.Object AS obj " + \
        "WHERE (obj.detect_isPrimary = 1) AND " + \
        "(obj.i_cModelFlux/obj.i_cModelFluxErr > 20) AND " + \
        "(obj.i_extendedness = 1) AND " + \
        "(obj.i_kronFlux_flag = 0) AND (obj.i_cModel_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS',"+str(random_RA)+","+str(random_dec)+", 0.1)) = 1 "

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

results = job.fetch_result()
tab2 = results.to_table()


First, calculate the magnitudes of galaxies in the 'random' location (outside the galaxy cluster) and also the magnitudes for the other filters near the galaxy cluster from the query performed in Section 2. This will enable plotting their colors.

> **Warning:** Like in Section 2.3, the following cell will produce warnings for invalid value encountered in log10, which happens if the source flux is negative. This happens for a small number of objects and since the goal of the plot is to see the distribution of the majority of sources, the warning can be safely ignored. 

In [ ]:
# skip/delete
i_random_gaap_mag = -2.50 * np.log10(tab2['i_gaapOptimalFlux']) + 31.4
r_random_gaap_mag = -2.50 * np.log10(tab2['r_gaapOptimalFlux']) + 31.4
g_random_gaap_mag = -2.50 * np.log10(tab2['g_gaapOptimalFlux']) + 31.4

u_cluster_gaap_mag = -2.50 * np.log10(tab['u_gaapOptimalFlux']) + 31.4
g_cluster_gaap_mag = -2.50 * np.log10(tab['g_gaapOptimalFlux']) + 31.4
r_cluster_gaap_mag = -2.50 * np.log10(tab['r_gaapOptimalFlux']) + 31.4
i_cluster_gaap_mag = -2.50 * np.log10(tab['i_gaapOptimalFlux']) + 31.4


In [ ]:
fig, (ax, ax1) = plt.subplots(ncols=1, nrows=2, figsize=(10, 6))

ax.plot(i_random_gaap_mag, (g_random_gaap_mag-i_random_gaap_mag),
        '.', alpha=.1, color='blue', label='Field Galaxies')
ax.set_xlabel('i-band Magnitude [GaaP]')
ax.set_ylabel('g-i color')
ax.set_ylim([-1, 4])
ax.legend()

ax1.plot(i_cluster_gaap_mag, (g_cluster_gaap_mag-i_cluster_gaap_mag),
         '.', alpha=.1, color='red', label='Cluster Galaxies')
ax1.set_xlabel('i-band Magnitude [GaaP]')
ax1.set_ylabel('g-i color')
ax1.set_ylim([-1, 4])
ax1.legend()

> Figure 6: 

A very nice red sequence appears from the red, old galaxies in the cluster! 